In [1]:
import random

from deap import base
from deap import creator
from deap import tools

from pprint import pprint

import numpy

In [2]:
TAM_CROMOSSOMO = 64
TAM_POPULACAO = 100

def fitness(cromossomo):
    """Funcao fitness para maximizar o cromossomo
    Deve retornar uma tupla!!!
    """
    count=0
    for i in range(TAM_CROMOSSOMO):
        if cromossomo[i] == 1:
            count += 1
            
    # Máximo 8 rainhas por cromossomo
    if (count != 8):
        return 99999,

    for i in range(TAM_CROMOSSOMO):
        # Separa o tabuleiro por linhas
        fin = 8
        ant = 0
        tabuleiro = []
        for j in range(8):
            tabuleiro.append(cromossomo[ant:fin])
            ant = fin
            fin += 8

    return avalia_tabuleiro(tabuleiro),

def ajusta_pop_ini(populacao, tam_populacao, tam_cromossomo):
    for i in range(tam_populacao):
        for j in range(tam_cromossomo):
            # Zera o cromossomo gerado
            populacao[i][j] = 0

    for i in range(tam_cromossomo):
        # Gera as 8 rainhas desse cromossomo
        fin = 7
        ant = 0
        for _ in range(8):
            # Uma rainha por linha
            col = random.randint(ant, fin)
            while (col > TAM_CROMOSSOMO):
                col = random.randint(ant, fin)

            populacao[i][col] = 1
            ant = fin
            fin += 8

def avalia_tabuleiro(tabuleiro):
    fit = 0

    # Avalia se tem uma rainha por linha
    for i in range(8):
        qtdRainha = 0

        for j in range(8):
            if tabuleiro[i][j] == 1:
                # Rainhas por linha
                qtdRainha += 1

                k = i + 1
                while (k <= 7):
                    # Rainhas por coluna
                    if tabuleiro[k][j] == 1:
                        qtdRainha += 1
                    k += 1

        # Se não houver ou houver mais de uma, deve descartar a solução
        if qtdRainha == 0:
            return 9999
        if qtdRainha > 1:
            fit += qtdRainha

    # Avalia as diagonais
    for i in range(8):
       for j in range(8):
           if tabuleiro[i][j] == 1:
                fit += avalia_diagonal(i, j, tabuleiro)

    return fit

def avalia_diagonal(x, y, tabuleiro):
    fit = 0

    # Mapeia as linhas abaixo e a direita
    newX = x + 1
    newY = y + 1
    while(newX <= 7 and newY <= 7):
        fit += tabuleiro[newX][newY]
        newX += 1
        newY += 1

    # Mapeia as linhas abaixo e a esquerda
    newX = x + 1
    newY = y - 1
    while(newX <= 7 and newY >= 0):
        fit += tabuleiro[newX][newY]
        newX += 1
        newY -= 1

    # Mapeia as linhas acima e a direita
    newX = x - 1
    newY = y + 1
    while (newX >= 0 and newY <= 7):
        fit += tabuleiro[newX][newY]
        newX -= 1
        newY += 1

    # Mapeia as linhas acima e a esquerda
    newX = x - 1
    newY = y - 1
    while (newX >= 0 and newY >= 0):
        fit += tabuleiro[newX][newY]
        newX -= 1
        newY -= 1

    return fit

In [3]:
def varAnd(population, toolbox, cxpb, mutpb):
    offspring = [toolbox.clone(ind) for ind in population]

    # Apply crossover and mutation on the offspring
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                          offspring[i])
            del offspring[i - 1].fitness.values, offspring[i].fitness.values

    for i in range(len(offspring)):
        if random.random() < mutpb:
            offspring[i], = toolbox.mutate(offspring[i])
            del offspring[i].fitness.values

    return offspring

def eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print (logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Gera os filhos
        offspring = toolbox.select(population, len(population))

        # Mutação
        offspring = varAnd(offspring, toolbox, cxpb, mutpb)

        # Avalia os fitness inválidos
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Seleciona usando o algoritmo usado nos filhos
        total = population + offspring
        population = toolbox.select(total, int(len(total)/2))

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print (logbook.stream)

    return population, logbook

In [4]:
# Define a estrategia do fitness
# - weights: define se o problema é de maximizacao (+1) ou minimizacao (-1)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))

# Define a estrutura do cromossomo
creator.create("Individual", list, fitness=creator.FitnessMax)

In [5]:
# Define os componentes para configurar a populacao
toolbox = base.Toolbox()

# Gerador para os individuos
toolbox.register("attr_bool", random.randint, 0, 1)

# Inicializador da populacao
toolbox.register("individual", 
                 tools.initRepeat, 
                 creator.Individual, 
                 toolbox.attr_bool, TAM_CROMOSSOMO)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [6]:
# Define os operadores geneticos
toolbox.register("evaluate", fitness)

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [7]:
# Cria a populacao inicial
populacao = toolbox.population(n=TAM_POPULACAO)
ajusta_pop_ini(populacao, TAM_POPULACAO, TAM_CROMOSSOMO)

In [8]:
hof = tools.HallOfFame(10)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)
    
#pop, log = algorithms.eaSimple(populacao, 
pop, log = eaSimple(populacao, 
                               toolbox, 
                               cxpb=1, 
                               mutpb=0.1, 
                               ngen=500, 
                               stats=stats, 
                               halloffame=hof, 
                               verbose=True)

gen	nevals	avg    	std    	min	max  
0  	100   	44602.6	46330.6	8  	99999
1  	100   	9209.66	25242.1	8  	99999
2  	100   	2911.38	14160.4	7  	99999
3  	100   	1210.44	10026.6	7  	99999
4  	100   	8.97   	1.45914	6  	14   
5  	100   	8.08   	1.69517	6  	16   
6  	100   	7.05   	0.779423	6  	10   
7  	100   	1006.52	9949.12 	4  	99999
8  	100   	6.31   	1.19746 	6  	16   
9  	100   	6.07   	0.430232	6  	10   
10 	100   	5.92   	0.79599 	2  	10   
11 	100   	5.72   	1.02059 	2  	6    
12 	100   	4.88   	1.796   	2  	6    
13 	100   	2.84   	1.62923 	2  	6    
14 	100   	2.04   	0.397995	2  	6    
15 	100   	2      	0       	2  	2    
16 	100   	2      	0       	2  	2    
17 	100   	2      	0       	2  	2    
18 	100   	2      	0       	2  	2    
19 	100   	2      	0       	2  	2    
20 	100   	2      	0       	2  	2    
21 	100   	2      	0       	2  	2    
22 	100   	2      	0       	2  	2    
23 	100   	2      	0       	2  	2    
24 	100   	2      	0       	2  	2    
25 	100   	2      	

223	100   	2      	0       	2  	2    
224	100   	2      	0       	2  	2    
225	100   	2      	0       	2  	2    
226	100   	2      	0       	2  	2    
227	100   	2      	0       	2  	2    
228	100   	1001.97	9949.58 	2  	99999
229	100   	2      	0       	2  	2    
230	100   	2      	0       	2  	2    
231	100   	2      	0       	2  	2    
232	100   	2      	0       	2  	2    
233	100   	2      	0       	2  	2    
234	100   	2      	0       	2  	2    
235	100   	2      	0       	2  	2    
236	100   	2      	0       	2  	2    
237	100   	2      	0       	2  	2    
238	100   	2      	0       	2  	2    
239	100   	2      	0       	2  	2    
240	100   	2      	0       	2  	2    
241	100   	2      	0       	2  	2    
242	100   	2      	0       	2  	2    
243	100   	2      	0       	2  	2    
244	100   	2      	0       	2  	2    
245	100   	2      	0       	2  	2    
246	100   	2      	0       	2  	2    
247	100   	2      	0       	2  	2    
248	100   	2      	0       	2  	2    
249	100   	2

447	100   	2      	0       	2  	2    
448	100   	2      	0       	2  	2    
449	100   	2      	0       	2  	2    
450	100   	2      	0       	2  	2    
451	100   	2      	0       	2  	2    
452	100   	1001.97	9949.58 	2  	99999
453	100   	2      	0       	2  	2    
454	100   	2      	0       	2  	2    
455	100   	2      	0       	2  	2    
456	100   	2      	0       	2  	2    
457	100   	2      	0       	2  	2    
458	100   	2      	0       	2  	2    
459	100   	2      	0       	2  	2    
460	100   	2      	0       	2  	2    
461	100   	2      	0       	2  	2    
462	100   	2      	0       	2  	2    
463	100   	2      	0       	2  	2    
464	100   	2      	0       	2  	2    
465	100   	2      	0       	2  	2    
466	100   	2      	0       	2  	2    
467	100   	2      	0       	2  	2    
468	100   	2      	0       	2  	2    
469	100   	2      	0       	2  	2    
470	100   	2      	0       	2  	2    
471	100   	2      	0       	2  	2    
472	100   	1001.97	9949.58 	2  	99999
473	100   	2

In [9]:
melhor = sorted([(x, x.fitness.values) for x in pop], key=lambda x: x[1])[0][0]

fin = 8
ant = 0
for i in range(8):
    pprint(melhor[ant:fin])
    ant = fin
    fin += 8

[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
